In [5]:
import requests
from datetime import datetime, timedelta

BASE_URL = "https://api.ret.nl/v2/departures"  # replace with real endpoint
STOP_ID = "SCHIEKADE"  # replace with real stop identifier

def get_future_departures(hours_ahead: int = 1):
    # Europe/Amsterdam "naïve" example; for perfect TZ use pytz/zoneinfo
    future_dt = datetime.now() + timedelta(hours=hours_ahead)
    date_str = future_dt.strftime("%Y-%m-%d")
    time_str = future_dt.strftime("%H:%M")

    params = {
        "stopId": STOP_ID,
        "date": date_str,
        "time": time_str,
        # add any extra params you see in DevTools (direction, maxResults, etc.)
    }

    try:
        resp = requests.get(BASE_URL, params=params, timeout=10)
        resp.raise_for_status()
        data = resp.json()
        return data
    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 403:
            print(f"❌ 403 Forbidden: Access denied. Check API key or authentication.")
            print(f"Response: {e.response.text}")
            return None
        else:
            raise

if __name__ == "__main__":
    departures = get_future_departures(hours_ahead=2)
    if departures:
        print(departures)
    else:
        print("Failed to retrieve departures.")

❌ 403 Forbidden: Access denied. Check API key or authentication.
Response: <html>
<head><title>403 Forbidden</title></head>
<body>
<center><h1>403 Forbidden</h1></center>
<hr><center>Microsoft-Azure-Application-Gateway/v2</center>
</body>
</html>

Failed to retrieve departures.


In [7]:
import re
import json

def get_modal_data_from_ret_page(url: str):
    """
    Fetch the RET page and extract the modalData JavaScript variable.
    """
    try:
        resp = requests.get(url, timeout=10)
        resp.raise_for_status()
        html_content = resp.text
        
        # Look for var modalData = {...} within <script> tags
        # Use non-greedy match and handle nested braces
        pattern = r'var\s+modalData\s*=\s*(\{[\s\S]*?\n\s*\})\s*(?:</script>|$)'
        match = re.search(pattern, html_content)
        
        if match:
            modal_data_str = match.group(1)
            # Clean up any trailing commas before closing braces (invalid JSON)
            modal_data_str = re.sub(r',(\s*[}\]])', r'\1', modal_data_str)
            modal_data = json.loads(modal_data_str)
            return modal_data
        else:
            print("❌ Could not find modalData variable in the HTML")
            # Try to find the script tag for debugging
            script_match = re.search(r'<script>[\s\S]*?var\s+modalData[\s\S]*?</script>', html_content)
            if script_match:
                print("Found script tag but couldn't parse. First 500 chars:")
                print(script_match.group(0)[:500])
            return None
            
    except requests.exceptions.RequestException as e:
        print(f"❌ Request failed: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"❌ Failed to parse JSON: {e}")
        print(f"Problematic JSON (first 500 chars):\n{modal_data_str[:500]}")
        return None

# Test with Schiekade
url = "https://www.ret.nl/home/reizen/halte/schiekade.html"
modal_data = get_modal_data_from_ret_page(url)

if modal_data:
    print("✅ Successfully extracted modalData:")
    print(json.dumps(modal_data, indent=2))
    print(f"\n📊 Found {len(modal_data)} timetable entries")
    for key, value in modal_data.items():
        if isinstance(value, dict) and 'title' in value:
            print(f"  - {value['title']} → {value.get('direction', 'Unknown')}")
else:
    print("Failed to extract modalData")

✅ Successfully extracted modalData:
{
  "timetable-overview-451": {
    "id": "timetable-overview-451",
    "title": "Tram 8",
    "direction": "Schiebroek",
    "mapID": "c18805f80be7406cbe310240d34868c6",
    "mapTitle": "Kaart",
    "latitude": "51.927987234863",
    "longitude": "4.4743492479239",
    "tooltipDefault": "Sla deze lijn op als favoriet",
    "tooltipSaved": "Opgeslagen",
    "tooltipDelete": "Verwijderd",
    "ariaLabel": "Favorieten",
    "favorite": "",
    "favourite": {
      "id": "451",
      "title": "Tram 8",
      "direction": "Schiebroek",
      "stop": "Tram 8",
      "urls": {
        "bus": "/home/reizen/dienstregeling/tram-8.html",
        "stop": "halte.html",
        "stopid": "205816087",
        "stopcode": "NL:S:31001062",
        "chbquay": "NL:Q:31001095",
        "chbstopplace": "NL:S:31001062"
      }
    },
    "departure": {
      "default": "11:04",
      "relative": "5"
    },
    "times": [
      {
        "default": "11:04",
        "relat

In [8]:
import requests
import json
from bs4 import BeautifulSoup

def parse_departures_from_html(html_content: str):
    """
    Parse the departure information from the HTML content.
    """
    soup = BeautifulSoup(html_content, 'html.parser')
    departures = []
    
    # Find all departure rows
    departure_rows = soup.find_all('a', class_='modal__toggle--generated')
    
    for row in departure_rows:
        departure = {}
        
        # Extract line name (e.g., "Tram 8")
        line_info = row.find('span', class_='favorite__info')
        if line_info:
            departure['line'] = line_info.get_text(strip=True)
        
        # Extract direction
        direction_div = row.find('div', class_='favorite__stop')
        if direction_div:
            direction_span = direction_div.find_all('span', class_='favorite__info')
            if direction_span:
                # Get text and clean up (remove SVG content)
                direction_text = direction_span[-1].get_text(strip=True)
                departure['direction'] = direction_text
        
        # Extract departure time
        time_spans = row.find_all('span', class_='favorite__time__amount')
        if len(time_spans) >= 1:
            departure['departure_time'] = time_spans[0].get_text(strip=True)
        
        # Extract minutes until departure
        minutes_span = row.find('span', class_='favorite__time__amount minutes')
        if minutes_span:
            departure['minutes'] = minutes_span.get_text(strip=True)
        
        # Extract data attributes
        departure['stop_line_code'] = row.get('data-stop-line-code', '')
        departure['stop_place_code'] = row.get('data-stop-place-code', '')
        departure['ride_date'] = row.get('data-ride-date', '')
        departure['ride_name'] = row.get('data-ride-name', '')
        departure['modal_id'] = row.get('data-modal', '')
        
        departures.append(departure)
    
    return departures

# Fetch and parse both modalData and HTML departures
url = "https://www.ret.nl/home/reizen/halte/schiekade.html"

try:
    resp = requests.get(url, timeout=10)
    resp.raise_for_status()
    html_content = resp.text
    
    # Parse HTML departures
    departures = parse_departures_from_html(html_content)
    
    print(f"✅ Found {len(departures)} departures from HTML:")
    print(json.dumps(departures, indent=2, ensure_ascii=False))
    
    print("\n" + "="*60)
    print("Summary of departures:")
    print("="*60)
    for dep in departures:
        print(f"{dep.get('line', 'Unknown')} → {dep.get('direction', 'Unknown')}")
        print(f"  Departure: {dep.get('departure_time', 'N/A')} (in {dep.get('minutes', 'N/A')} min)")
        print(f"  Stop code: {dep.get('stop_line_code', 'N/A')}")
        print()
        
except Exception as e:
    print(f"❌ Error: {e}")

✅ Found 2 departures from HTML:
[
  {
    "line": "Tram 8",
    "direction": "Schiebroek",
    "departure_time": "11:04",
    "minutes": "5",
    "stop_line_code": "NL:Q:31001095",
    "stop_place_code": "NL:Q:31001095",
    "ride_date": "2025-11-16",
    "ride_name": "915721",
    "modal_id": "timetable-overview-451"
  },
  {
    "line": "Tram 8",
    "direction": "Spangen",
    "departure_time": "11:12",
    "minutes": "13",
    "stop_line_code": "NL:Q:31001062",
    "stop_place_code": "NL:Q:31001062",
    "ride_date": "2025-11-16",
    "ride_name": "916681",
    "modal_id": "timetable-overview-450"
  }
]

Summary of departures:
Tram 8 → Schiebroek
  Departure: 11:04 (in 5 min)
  Stop code: NL:Q:31001095

Tram 8 → Spangen
  Departure: 11:12 (in 13 min)
  Stop code: NL:Q:31001062

